# Lesson 2: Chain-of-Thought and ReACT Prompting

## Demand‑Spike Detective for Retail Promotions

In this hands-on exercise, you will guide an LLM to explain an unexpected sales spike--probing promotions data, competitor-pricing changes, and weather shifts--and then recommend the right inventory response for the retail chain.
Outline:

- Setup
- Understand SKU‑level sales data, promotional calendars, etc. 
- Craft a CoT prompt that has the LLM hypothesize causes based on data provided.
- Create a ReACT prompt that can call a (simulated) weather‑API tool when the model suspects weather as a driver. 

## 1. Setup

Let's start by setting up the environment.

In [ ]:
# Import necessary libraries
# No changes needed in this cell

from openai import OpenAI
from IPython.display import Markdown, display
from enum import Enum
import os
from lesson_2_lib import (
    get_competitor_pricing_data,
    get_sales_data,
    get_promotions_data,
    get_weather_data,
    call_weather_api,
    print_in_box,
)
import pandas as pd


In [ ]:
# If using the Vocareum API endpoint
# TODO: Fill in the missing parts marked with **********

client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    # api_key="**********",  # <--- TODO: Fill in your Vocareum API key here
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)

# If using OpenAI's API endpoint
# client = OpenAI()


In [ ]:
# Set up helper functions
# No changes needed in this cell


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_MINI


def get_completion(system_prompt, user_prompt, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def get_completion_messages(messages, model=MODEL, temperature=0.1):
    # Check that all messages are of the right form
    for message in messages:
        if (
            not isinstance(message, dict)
            or "role" not in message
            or "content" not in message
            or message["role"] not in ["system", "user", "assistant"]
        ):
            raise ValueError(f"Message is not valid: {message!r}")
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def display_responses(*args, user_prompt_limit=500):
    """Helper function to display responses as Markdown, horizontally."""
    markdown_string = "<table><tr>"
    # Headers
    for arg in args:
        markdown_string += f"<th>System Prompt:<br />{arg['system_prompt']}<br /><br />"
        markdown_string += f"User Prompt:<br />{arg['user_prompt'][:user_prompt_limit]}"
        if len(arg["user_prompt"]) > user_prompt_limit:
            markdown_string += "... [truncated]"
        markdown_string += "</th>"
    markdown_string += "</tr>"
    # Rows
    markdown_string += "<tr>"
    for arg in args:
        markdown_string += f"<td>Response:<br />{arg['response']}</td>"
    markdown_string += "</tr></table>"
    display(Markdown(markdown_string))

## 2. Understand SKU-level sales data, promotional calendars, etc.

First, let's review the sample data provided. Working with AI Agents is still a data problem at its core, so the first steps are always to understand the business goals (explain the cause for the spike) and the underlying data.

In [ ]:
# Load the simulated data
# No changes needed in this cell

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

sales_data = get_sales_data()
sales_df = pd.DataFrame(sales_data)

promotions_data = get_promotions_data()
promotions_df = pd.DataFrame(promotions_data)

weather_data = get_weather_data()
weather_df = pd.DataFrame(weather_data)

competitor_pricing_data = get_competitor_pricing_data()
competitor_pricing_df = pd.DataFrame(competitor_pricing_data)


In [ ]:
# Show the sales data
# No changes needed in this cell

sales_df = sales_df.sort_values(by=["product_id", "date"]).reset_index(drop=True)
sales_df


In [ ]:
# Show the promotions data
# No changes needed in this cell

promotions_df

In [ ]:
# Show the weather data
# No changes needed in this cell

weather_df.head()

In [ ]:
# Show the competitor pricing data
# No changes needed in this cell

competitor_pricing_df

In [ ]:
# Graph the sales data
# No changes needed in this cell

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
for product_id, product_data in sales_df.groupby("product_id"):
    product_data.sort_values(by="date", inplace=True)
    plt.plot(product_data["date"], product_data["quantity"], label=product_id)
plt.xlabel("Date")
plt.ylabel("Quantity")
plt.title("Sales Data for Each Product")
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Plot the weather data

weather_df["temperature_c"] = weather_df["temperature"].apply(lambda x: x["celsius"])


plt.figure(figsize=(6, 4))
plt.plot(weather_df["date"], weather_df["temperature_c"])

# For each day, add the value of "main" in text
for i, row in weather_df.iterrows():
    # Add a transparent background to the text
    plt.text(
        row["date"],
        row["temperature_c"],
        row["conditions"]["main"],
        backgroundcolor="white",
    )


plt.xlabel("Date")
plt.xticks(rotation=45)
plt.ylabel("Temperature")
plt.title("Weather Data for Each Day")
plt.show()

# Plot the competitor pricing data

plt.figure(figsize=(6, 4))
for competitor in ["a", "b", "c"]:
    for product in competitor_pricing_df["product"].unique():
        plt.plot(
            competitor_pricing_df[competitor_pricing_df["product"] == product]["date"],
            competitor_pricing_df[competitor_pricing_df["product"] == product][
                f"competitor_{competitor}_price"
            ],
            label=f"{product} - competitor {competitor.upper()}",
            color=f"C{product[-1]}",
        )
plt.xlabel("Date")
plt.xticks(rotation=45)
plt.ylabel("Price")
plt.title("Competitor Prices for Each Product")
# Put the legend outside the plot
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

### Reflect on the sales data

`TODO: Insert your reflection here`

Reflection: The sales data shows a clear pattern of sales spikes for Product 5 on January 12th, which is the same day as a promotion. There was a promotion, "Weekend Special", which started on January 12th, but that was for Product 2.

Looking ahead we see that the weather was fluctuating between below freezing and above freezing. On the 12th in particular, there was heavy rain.

Competitor pricing is a bit harder to look at, but if we isolate it to Product 5, we see that the price for one competitor dropped on January 12th, which may have led to more overall consumer interest in that product or may have coincided an ad campaign.

In practice, a human would look at all these factors and more to determine the cause of the sales spike by digging deeper, investigating all possible causes.

Let's see if we can get this far with a CoT prompt, which should at least find the sales spike and not assume this promotion is responsible.

## 2. Craft a simple CoT prompt

Let's start with a simple CoT prompt. We won't tell the model to which steps to follow. We also won't use any tools in this example.

<div style="color: red">Note: Modern LLMs such as GPT-4.1 may not need an explicit "think in steps" phrase or instructions to think in steps, as this behavior can be added to a model in its fine-tuning process. In fact, some models such as o1 may refuse to respond to prompts that ask for CoT. Consider running your prompts with and without asking for CoT explicitly.</div>


In [ ]:
# TODO: Fill in the part marked with ********** with a phrase such as "Think step by step."
# Feel free to try variations!

system_prompt_explicit_cot = """
You are a meticulous Retail Demand Analyst.
Your task is to analyze provided sales data and promotion schedules to identify and explain significant sales spikes for specific SKUs.

**********
"""

system_prompt_no_explicit_cot = """
You are a meticulous Retail Demand Analyst.
Your task is to analyze provided sales data and promotion schedules to identify and explain significant sales spikes for specific SKUs.
"""

user_prompt_analyze = f"""
Analyze the data provided below and hypothesize causes for any observed sales spikes.

Sales Data:
{sales_data}

Promotions Calendar:
{promotions_data}

Weather Data:
{weather_data}

Competitor Pricing Data:
{competitor_pricing_data}
"""

print(f"Sending prompt to {MODEL} model...")
explicit_cot_response_1 = get_completion(
    system_prompt_explicit_cot, user_prompt_analyze
)
no_explicit_cot_response_2 = get_completion(
    system_prompt_no_explicit_cot, user_prompt_analyze
)
print("Response received!\n")

# We compare the explicit CoT and non-ex
display_responses(
    {
        "system_prompt": system_prompt_explicit_cot,
        "user_prompt": user_prompt_analyze,
        "response": explicit_cot_response_1,
    },
    {
        "system_prompt": system_prompt_no_explicit_cot,
        "user_prompt": user_prompt_analyze,
        "response": no_explicit_cot_response_2,
    },
)


### Observation

- What did you notice about the output of this CoT prompt?
- What were the differences between including CoT and not including CoT explicitly in the prompt?

## 3. Craft a More Developed CoT Prompt

Let's add more to our CoT prompt. Let's ask the model to follow a specific set of steps in an instructions section. This may help us not only get final answers that align with our needs, but it will also help us constrain the output to a more specific format.

Finally, we'd like to the model to get the single largest spike and hypothesize its causes, using the following output format:

````

STRUCTURED ANALYSIS:
[Structured Analysis]

LARGEST SPIKE:
```json
{
    "date": "YYYY-MM-DD",
    "amount": "X.XX",
    "percentage": "X.XX%"
    "causes": [
        "Cause 1",
        "Cause 2",
        "Cause 3"
    ]
}
```

````

In [ ]:
# Let's add some more components to our CoT Prompt
# TODO: Replace parts marked with a **********

system_prompt_cot = """
You are a meticulous Retail Demand Analyst.
Your task is to analyze provided sales data and promotion schedules to identify and explain significant sales spikes for specific SKUs.

Think in steps.
"""

user_prompt_analyze = f"""
## INSTRUCTIONS:

Analyze the data provided below and hypothesize causes for any observed sales spikes.

Instructions:
* Find all sales spikes for each product
* For each product, identify the following:
    * Date of the sales spike
    * Amount of the sales spike and percentage increase
    * Possible causes of the sales spike according to the provided
        * sales data
        * promotion schedule
        * weather conditions
        * competitor pricing data
* Start with your analysis
* Conclude with the single largest spike according to the percentage increase with a short explanation for it.

--

## OUTPUT FORMAT:

STRUCTURED ANALYSIS:
[Structured Analysis]

LARGEST SPIKE:
```json
{{
    "date": "YYYY-MM-DD",
    "amount": "X.XX",
    "percentage": "X.XX%"
    "causes": [
        "Cause 1",
        "Cause 2",
        "Cause 3"
    ]
}}
```

---

## CONTEXT

Sales Data:
{sales_data}

Promotions Calendar:
{promotions_data}

Weather Data:
{weather_data}

Competitor Pricing Data:
{competitor_pricing_data}
"""

print(f"Sending prompt to {MODEL} model...")
cot_response = get_completion(system_prompt_cot, user_prompt_analyze)
print("Response received!\n")


def parse_analysis_and_largest_spike(response):
    import json

    if "LARGEST SPIKE:\n```json" not in response:
        print()
        print(response)
        raise RuntimeError(
            " ❌ No LARGEST SPIKE found in response. Looking for: \n\n LARGEST SPIKE:\n```json"
        )

    analysis = response.split("LARGEST SPIKE:\n```json")[0].strip()
    json_str = response.split("LARGEST SPIKE:\n```json")[1].split("```")[0].strip()
    return analysis, json.loads(json_str)


analysis, largest_spike = parse_analysis_and_largest_spike(cot_response)

display(Markdown(analysis))

In [ ]:
# Show the largest spike
# No changes needed in this cell

from pprint import pprint

pprint(largest_spike)


### Observation:

- Did the model follow your instructions?
- How was the description of the highest spike by percentage?

## 4. Create a ReACT prompt that can call a (simulated) weather‑API tool

While it is often convenient to throw all of the data into the prompt for the model to figure it out, sometimes the entire dataset is too large or too complex for the model to handle. In this case, we may want our model to be able to decide when to call a tool with what parameters.

When requesting the usage of a tool, the model will return a special output, signaling the orchestrator to call the tool (e.g. `tool_call: weather_api`). The orchestrator will then call the tool and put the result in the message history for the model to use.

Let's create a prompt that will have the model follow the ReACT pattern of Think, Act, Observe, and then repeat until it has a final answer.

For this exercise we will use the following tools:

### Available Tools
* `noop()`: Do nothing. Useful if you are not ready to take a tool call.
* `calculator(expression: str)`: Perform an arithmetic calculation
    - Example:
        - Input: `ACT: calculator(expression="(10 + 20) / 2.0")`
        - Output: `OBSERVE: 15.0`
* `get_sales_data()`: Get the sales data
    - Example:
        - Input: `ACT: get_sales_data()`
        - Output: `OBSERVE: {"date": "2024-01-10", "product_id": "P001", "product_name": "Product 1", "quantity": 255, "revenue": 15547.35}`
* `call_weather_api(date: str)`: Get weather data for a specific date. Call this for the date of each spike.
    - Example:
        - Input: `ACT: call_weather_api(date="2024-01-10")`
        - Output: `OBSERVE: {"date": "2024-01-10", "weather": "Sunny", "temperature": 72}`

* `final_answer(amount: str, causes: list[str], date: str, percentage: str)`: Return the final answer
    - Example:
        - Input: `ACT: final_answer(amount="32", causes=["Competitor X offering a 29 discount boosting category interest", ...], date="2020-06-12", percentage="20.00%")`
        - Output: `OBSERVE: {"amount": "32", "causes": ["Competitor X offering a 29 discount boosting category interest", ...], "date": "2020-06-12", "percentage": "20.00%"}`


In [ ]:
# First, let's create a ReACT prompt that will run for a single step.
# It should conclude with asking for a tool call.
# TODO: Replace parts marked with a **********


react_system_prompt = """
You are a meticulous Retail Demand Analyst that can solve any TASK in a multi-step process using tool calls and reasoning.

## Instructions:
- You will use step-by-step reasoning by
    - THINKING the next steps to take to complete the task and what next tool call to take to get one step closer to the final answer
    - ACTING on the single next tool call to take
- You will always respond with a single THINK/ACT message of the following format:
    THINK:
    **********
    ACT:
    **********
- As soon as you know the final answer, call the `final_answer` tool in an `ACT` message.
- ********** <--- Extra instructions


## Available Tools
**********

You will not use any other tools.

Example:

```
--USER MESSAGE--
TASK:
Respond to the query "What was the weather one week ago?". Today is 2024-01-17.

--ASSISTANT MESSAGE--
THINK:
********** <-- Finish the message history until the end of the loop
```
"""

user_prompt_analyze = """
TASK: Find the single largest sales spike according to the percentage increase with a short explanation for it
based on factors such as weather.
"""

print(f"Sending prompt to {MODEL} model...")

messages = []
messages.append({"role": "system", "content": react_system_prompt})
messages.append({"role": "user", "content": user_prompt_analyze})

react_response = get_completion_messages(messages)

messages.append({"role": "assistant", "content": react_response})
print("Response received!\n")


for message in messages:
    if message["role"] == "system":
        continue
    print_in_box(message["content"], title=f"{message['role'].capitalize()}")

assert "ACT:" in messages[-1]["content"], (
    " ❌ No ACT message found in response. Looking for: \n\n ACT:"
)

## 5. Tool Calling Parsing and Calling

Awesome! Let's now work on our functions that will parse the text following the `ACT:` part of the response and call a pre-defined function.

As a reminder, here are the tools we wish to support:

### Available Tools
* `noop()`: Do nothing. Useful if you are not ready to take a tool call.
* `calculator(expression: str)`: Perform an arithmetic calculation
    - Example:
        - Input: `ACT: calculator(expression="(10 + 20) / 2.0")`
        - Output: `OBSERVE: 15.0`
* `get_sales_data()`: Get the sales data
    - Example:
        - Input: `ACT: get_sales_data()`
        - Output: `OBSERVE: {"date": "2024-01-10", "product_id": "P001", "product_name": "Product 1", "quantity": 255, "revenue": 15547.35}`
* `call_weather_api(date: str)`: Get weather data for a specific date. Call this for the date of each spike.
    - Example:
        - Input: `ACT: call_weather_api(date="2024-01-10")`
        - Output: `OBSERVE: {"date": "2024-01-10", "weather": "Sunny", "temperature": 72}`

* `final_answer(amount: str, causes: list[str], date: str, percentage: str)`: Return the final answer
    - Example:
        - Input: `ACT: final_answer(amount="32", causes=["Competitor X offering a 29 discount boosting category interest", ...], date="2020-06-12", percentage="20.00%")`
        - Output: `OBSERVE: {"amount": "32", "causes": ["Competitor X offering a 29 discount boosting category interest", ...], "date": "2020-06-12", "percentage": "20.00%"}`

In [ ]:
# Let's work on our calculator function!
# TODO: Replace parts marked with a **********

import re

import ast
import operator


def safe_eval(expr):
    """
    Evaluate a mathematical expression safely.

    We normally don't want to use eval() because it can execute arbitrary code, unless we are in a
    properly sandboxed environment. This function is a safe alternative for evaluating mathematical
    expressions.
    """
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.USub: operator.neg,
    }

    def eval_node(node):
        if isinstance(node, ast.Constant):
            return node.value
        elif isinstance(node, ast.BinOp):
            return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
        elif isinstance(node, ast.UnaryOp):
            return operators[type(node.op)](eval_node(node.operand))
        elif isinstance(node, ast.Expr):
            return eval_node(node.value)
        else:
            raise TypeError(f"Unsupported type: {type(node)}")

    return eval_node(ast.parse(expr, mode="eval").body)


def calculator(expression: str) -> float:
    """
    Evaluate a mathematical expression safely.
    """
    return float(**********)  # TODO: Replace with a call to evaluate the expression


assert (actual := calculator("10 + 10")) == 20.0, f" ❌ Expected 20.0, got {actual}"

In [ ]:
def get_observation_message(response: str) -> str:
    """
    Take a THINK/ACT response, run the tool call, and return the observation message.

    Args:
        response (str): The THINK/ACT response.

    Returns:
        str: The observation message.

    Uses regular expressions to match the tool call and run the corresponding tool.

    If the response is invalid, return an error message as a string that the agent can understand.
    """
    from ast import literal_eval

    observation_message = None

    SALES_DATA_REGEX = r"ACT:\nget_sales_data\(\)"
    WEATHER_REGEX = r"ACT:\ncall_weather_api\(date=\"(.*)\"\)"
    CALCULATOR_REGEX = **********  # TODO: Add regex for calculator
    FINAL_ANSWER_REGEX = r"ACT:\nfinal_answer\(amount=\"(.*)\", causes=(.*), date=\"(.*)\", percentage=\"(.*)\"\)"
    NOOP_REGEX = **********  # TODO: Add regex for noop

    # TOOL 1: get_sales_data
    if re.search(SALES_DATA_REGEX, response):
        sales_data = get_sales_data(products=["P005"])
        # filter sales data to Product 5
        sales_data = [
            item for item in sales_data if item["product_name"] == "Product 5"
        ]
        observation_message = f"OBSERVE:\n{sales_data}"

    # TOOL 2: call_weather_api
    elif re.search(WEATHER_REGEX, response):
        date = re.search(WEATHER_REGEX, response).groups()[0]
        weather_data = call_weather_api(date)
        observation_message = f"OBSERVE:\n{weather_data}"

    # TOOL 3: calculator
    elif re.search(CALCULATOR_REGEX, response):
        expression = re.search(CALCULATOR_REGEX, response).groups()[0]
        observation_message = f"OBSERVE:\n{calculator(expression)}"

    # TOOL 4: final_answer
    elif re.search(FINAL_ANSWER_REGEX, response):
        amount, causes, date, percentage = re.search(
            FINAL_ANSWER_REGEX,
            response,
        ).groups()
        causes = literal_eval(causes)
        observation_message = f"OBSERVE:\namount: {amount}\ndate: {date}\npercentage: {percentage}\ncauses: {causes}"

    # TOOL 5: noop
    elif re.search(NOOP_REGEX, response):
        observation_message = "OBSERVE:\nNo action taken."

    # Error
    else:
        observation_message = "OBSERVE:\nInvalid tool call or tool not supported."

    return observation_message


# Test cases
assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
get_sales_data()
""")
) == (expected := "OBSERVE:\n" + str(get_sales_data(products=["P005"]))), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
call_weather_api(date="2024-01-12")
""")
) == (expected := "OBSERVE:\n" + str(call_weather_api("2024-01-12"))), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
final_answer(amount="10", causes=["cause1", "cause2"], date="2024-01-12", percentage="10%")
""")
) == (
    expected
    := "OBSERVE:\namount: 10\ndate: 2024-01-12\npercentage: 10%\ncauses: ['cause1', 'cause2']"
), f"{actual} != {expected}"

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
calculator(expression="10 + 10")
""")
) == (expected := "OBSERVE:\n20.0"), f"{actual} != {expected}"

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
noop()
""")
) == (expected := "OBSERVE:\nNo action taken."), f"{actual} != {expected}"

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
invalid_tool()
""")
) == (expected := "OBSERVE:\nInvalid tool call or tool not supported."), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT_TYPO:
get_sales_data()
""")
) == (expected := "OBSERVE:\nInvalid tool call or tool not supported."), (
    f"{actual} != {expected}"
)


## 6. Putting It All Together

Now we're ready to put it all together! We will now use the ReACT prompt we created in the previous section to call a (simulated) weather API tool. This will run in a loop for a maximum number of iterations until the `final_answer` tool is called.

In [ ]:
# Let's make the ReACT loop!
# TODO: Replace instances of ********** where specified

messages = []

# ********** <-- Add the react_system_prompt to the message history
# ********** <-- Add the user_prompt_analyze to the message history


for message in messages:
    if message["role"] == "system":
        continue
    print_in_box(message["content"], title=f"{message['role'].capitalize()}")

num_react_steps = 0

observation_message = None
while True:
    observation_message = None

    num_action_retries = 0

    while not observation_message:
        react_response = ********** # <-- Get the completion response from the current messages
        observation_message = ********** # <-- Call the tool and get the observation message
        if observation_message is None:
            print(f"ERROR: Retrying... {num_action_retries}")

        num_action_retries += 1
        if num_action_retries > 5:
            break

    messages.append({"role": "assistant", "content": react_response})
    print_in_box(
        react_response, title=f"Assistant (Think + Act). Step {num_react_steps + 1}"
    )

    messages.append({"role": "user", "content": observation_message})

    if "ACT:\nfinal_answer" in react_response:
        print_in_box(observation_message, title="FINAL ANSWER")
        break

    print_in_box(
        observation_message, title=f"User (Observe). Step {num_react_steps + 1}"
    )

    num_react_steps += 1
    if num_react_steps > **********:  # TODO: Add max number of React steps
        print("ERROR: Max number of React steps exceeded. Breaking.")
        break


## Reflection

Great work! Let's take a chance to think about what we've seen so far.

- What can you say about the need to explicitly state "Think in steps" in a prompt for using Chain of Thought reasoning?
- In what cases does using a single CoT prompt call work better than using a ReACT prompt and loop?
- In what cases does it work the other way around?
- In this case, which example was more efficient (faster, processed more data, etc.) and why?
- Suppose you wanted to see if the LLM actually needed the calculator in the ReACT example. How would you modify the ReACT prompt to try it without the calculator?

## Summary

🎉 Congratulations! 🎉 You've learned how to apply Chain-of-Thought and ReACT prompting techniques to retail analytics!

Through this exercise, you've seen how to:

- 📊 Guide an LLM through structured reasoning about complex retail data
- 🤔 Implement Chain-of-Thought prompting to analyze potential causes of sales spikes
- 🔄 Add ReACT components to incorporate external data validation

Keep up the good work! 💯